# SMARCC2

#### Clincal Synopsis:

Coffin-Siris syndrome-8 (CSS8) is characterized by variable degrees of impaired intellectual development including speech impairment, hypotonia, feeding difficulties, and behavioral abnormalities. Dysmorphic features may or may not be present and include hypertrichosis or thin scalp hair, thick eyebrows, thin upper vermilion, and upturned nose (Machol et al., 2019). 

For a general phenotypic description and a discussion of genetic heterogeneity of Coffin-Siris syndrome, see CSS1 (135900).



In [2]:
cohort_name = 'SMARCC2'
tx_id = 'NM_001330288'
px_id = 'NP_001317217.1'

In [5]:
import hpotk
store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo()

In [6]:
from ppktstore.registry import configure_phenopacket_registry
phenopacket_registry = configure_phenopacket_registry()
with phenopacket_registry.open_phenopacket_store('0.1.18') as ps:
    phenopackets = tuple(ps.iter_cohort_phenopackets(cohort_name))
len(phenopackets)

65

In [7]:
from gpsea.preprocessing import configure_caching_cohort_creator, load_phenopackets
cohort_creator = configure_caching_cohort_creator(hpo)
cohort, validation = load_phenopackets(  
    phenopackets=phenopackets,
    cohort_creator=cohort_creator,
)

Individuals Processed:   0%|          | 0/65 [00:00<?, ?individuals/s]